<a href="https://colab.research.google.com/github/LaertesPecker/Clasificadores/blob/master/Practica1_Reconocimiento_Formas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRÁCTICA 1: Clasificadores Euclídeo y Bayesiano


---



## 1. Introducción



A lo largo de esta práctica veremos como construir diversos clasificadores. También veremos cuándo y por qué son útiles así como medir sus rendimientos. Finalmente, aplicaremos los conocimientos adquiridos en un problema real y veremos la potencia que tienen estos sencillos clasificadores.

## 2. Clasificador de la distancia euclídea

### 2.1 En qué consiste el clasificador y como lo hemos implementado

El clasificador de la distancia euclídea parte de la hipótesis de que dispersion de las clases es pequeña en relación a la distancia entre ellas. Una vez aceptada la hipótesis, el representante de cada clase será el centroide de todos los datos pertenecientes a esa clase. Por tanto, el primer paso para impementar en clasificador será calcular los centroides de los datos de test. Una vez calculados los centroides, para determinar la pertenencia de un dato a una clase, calcularemos la distancia del dato a todos los centroides y el dato pertenecerá a la clase que representa el centroide más cercano a él. 

Matemáticamente, el cálculo del centroide es: $z_i = \frac{1}{card(\alpha_i)}\sum_{x \in \alpha_i} x$, donde $\alpha_i$ representa a la clase i. Como ya se ha expuesto arriba, la función discriminante será la distancia euclídea $f_i(x)=d_E(x,z_i)$. Por tanto, se clasificará un objeto en la clase cuya distancia
sea la mínima, es decir $x \in \alpha_i \leftrightarrow arg min_i{f_i(x)}$

### 2.2 Implementación del clasificador

Para implementar el clasificador se deberán definir 4 funciones: La función de **fit**, la cual calculará los centroides de nuestros datos de test. La función **predict**, la cual calculará la distancia de todos nuestros datos los cuales queremos obtener su clase a todos los centroides calculados en la función fit. La función **pred_label**, que cogerá la etiqueta del centroide mas cercano a todos los datos que queremos predecir. Finalmente, la función **num_aciertos**, que calculará el número de aciertos de nuestro clasificador.

Empezamos importando los paquetes necesarios y creando la clase y el constructor de la misma:

In [0]:
import numpy as np
from abc import abstractmethod

class Classifier:
    @abstractmethod
    def fit(self,X,y):
        pass

    @abstractmethod
    def predict(self,X):
        pass

class ClassifEuclid(Classifier):
    def __init__(self,labels=[]):
        self.labels=labels
        pass

A continuación, definimos la función fit:

In [0]:
def fit(self,X,y):
        self = np.array([np.mean(X[y==l],axis=0) for l in np.unique(y)])
        return self

Como se ha descrito arriba, la función fit calcula los centroides, para ello recorre todas las x con misma etiqueta y calcula su media. A continuación se procede a escribir la función predict:

In [0]:
def predict(self,X):
        distancias = np.linalg.norm(centroides[:,np.newaxis] - X,axis=2)
        return distancias

Calcula la norma entre los centroides y los valores de x, es decir, calcula la distancia euclídea y la almacena en la variable distancias. Hay que añadiendo una nueva columna o hacer un reshape de centroides para poder operar con x, haciendo el reshape para los datos del iris deberíamos cambiar la matriz de centroides con:

```
centroides.reshape(3,1,4)
```
Continuamos definiendo la función pred_label:


In [0]:
def pred_label(self,X):
        distancias = predict(self,X)
        pred = np.argmin(distancias,axis=0)
        return pred

Finalmente, calculamos el número de aciertos y el porcentaje de aciertos de nuestro clasificador:

In [0]:
def num_aciertos(self,X,y):
        pred = pred_label(self,X)
        numAciertos = (pred == y).sum()
        porcAciertos = (pred==y).mean()*100
        print(f"Numero de aciertos: {numAciertos} , Porcetaje de Aciertos: {porcAciertos}" )
        return

Definimos una función más que utilizaremos cuando queramos utilizar una parte de los datos para entrenar nuestro clasificador, por defecto entrenaremos con el 70% de los datos y testearemos con el otro 30%.

In [0]:
def clasfEucl(X,y,name,test_size=0.3):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
  centroides = np.array([np.mean(X_train[y_train==l],axis=0) for l in np.unique(y_train)])
  distancias = np.linalg.norm(centroides[:,np.newaxis] - X_test,axis=2)
  pred = np.argmin(distancias,axis=0)
  numAciertos = (pred == y_test).sum()
  porcAciertos = (pred == y_test).mean()*100
  print(f"Datos " + name + ":")
  print(f"Numero de aciertos: {numAciertos} , Porcetaje de Aciertos: {porcAciertos}" )

In [0]:
import numpy as np
import pandas as pd
from sklearn.covariance import ShrunkCovariance
from sklearn.base import BaseEstimator
from sklearn import preprocessing
from abc import abstractmethod


class Classifier(BaseEstimator):

    @abstractmethod
    def fit(self, X, y):
        pass

    @abstractmethod
    def predict(self, X):
        pass


class ClassifEuclid(Classifier):
    def __init__(self):
        """Constructor de la clase
        labels: lista de etiquetas de esta clase"""
        self.centroids = None

    def fit(self, X, y):
        """Entrena el clasificador
        X: matriz numpy cada fila es un dato, cada columna una medida
        y: vector de etiquetas, tantos elementos como filas en X
        retorna objeto clasificador"""
        assert X.ndim == 2 and X.shape[0] == len(y)
        
        # Entrena el clasificador
        self.centroids = np.array([np.mean(X[y==l],axis=0) for l in np.unique(y)])
        return self
        
        return self

    def decision_function(self, X):
        """Estima el grado de pertenencia de cada dato a todas las clases
        X: matriz numpy cada fila es un dato, cada columna una medida del vector de caracteristicas.
        Retorna una matriz, con tantas filas como datos y tantas columnas como clases tenga
        el problema, cada fila almacena los valores pertenencia de un dato a cada clase"""
        assert self.centroids is not None, "Error: The classifier needs to be fitted. Please call fit(X, y) method."
        assert X.ndim == 2 and X.shape[1] == self.centroids.shape[1]

        # Calcula y devuelve la distancia a cada centroide
        distancias = np.linalg.norm(self.centroids[:,np.newaxis] - X,axis=2)
        return distancias
    def predict(self, X):
        """Estima la etiqueta de cada dato. La etiqueta puede ser un entero o bien un string.
        X: matriz numpy cada fila es un dato, cada columna una medida
        retorna un vector con las etiquetas de cada dato"""
        # Calcula y devuelve la clase más probable
        distancias = self.decision_function(X)
        pred = np.argmin(distancias,axis=0)
        return pred
    def score (self,X,y):
        self.fit(X,y)
        pred = self.predict(X)
        porcAciertos = (pred==y).mean()*100
        return porcAciertos

### 2.3 Tabla de resultados. Discusión

#### 2.3.1 Tabla de resultados

Utilizamos nuestro clasificador para los datos de iris, wine y cancer. Para entrenar nuestro clasificador utilizaremos el 70% de los datos:

In [34]:
from sklearn.datasets import  load_iris, load_wine, load_breast_cancer
from sklearn.model_selection import train_test_split
X, y  = load_iris(return_X_y=True)
clasfEucl(X,y,"Iris",0.5)

Datos Iris:
Numero de aciertos: 72 , Porcetaje de Aciertos: 96.0


In [35]:
X, y  = load_wine(return_X_y=True)
clasfEucl(X,y,"Wine",0.5)

Datos Wine:
Numero de aciertos: 63 , Porcetaje de Aciertos: 70.78651685393258


In [36]:
X, y =load_breast_cancer(return_X_y=True)
clasfEucl(X,y,"Cancer",0.5)

Datos Cancer:
Numero de aciertos: 243 , Porcetaje de Aciertos: 85.26315789473684


Medida |Iris | Wine | Cancer
--- |--- | --- | ---
Numero Aciertos|42 | 42 | 155
Porcentaje Aciertos|92% | 70% | 90%


#### 2.3.2 Discusión



Como se puede observar a partir de la tabla, las bases de datos de iris y cancer obtienen muy buenos resultados de alrededor del 90% de aciertos. No obstante wine obtiene un porcentaje de aciertos del 70% aproximadamente. Esto puede ser debido a que los datos wine tienen una dispersión de las clases mayor en relación con la distancia entre ellas que la que pudiera haber en los datos del iris o del cancer

## 3. Clasificador Estadístico Bayesiano


### 3.1 En qué consiste el clasificador y como lo hemos implementado


La medida de pertenencia ahora va a estar establecida por la estadística bayesiana. Cuando usabamos un clasificador euclídeo, la dispersión debía ser pequeña dentro de una clase para que funcionara bien. En este tema vamos a resolver el problema de clasificación mediante el modelado estadístico de la distribución de las muestras en casa clase. Este clasificador solo funcionará bien para clases que se distribuyan unimodalmente. Otro inconveniente de este clasificador es que el número de parámetros crece cuadráticamente. No obstante, este clasificador es muy simple matemáticamente y se adecua a muchos problemas. A menudo, es imposible encontrar una frontera que separe perfectamente dos clases. Vamos a minimizar este error con la estadística.

Para nuestro clasificador usaremos la probabilidad a posteriori pero, gracias al teorema de Bayes, conociendo las demás podemos conocer la a posteriori: 

$P(\alpha_i|x) = \frac{p(x|\alpha_i)P(\alpha_i)}{p(x)}$

Una forma de clasificar, por ejemplo, seria asignarle la que tenga mayor numerador. Por tanto, la funcion discriminante la podremos escribir como:
    
$f_j(x) = P(\alpha_j|x)$

$f_j(x) = p(\alpha_j,x) = p(x|\alpha_j)P(\alpha_j)$ (conjunta)

Ambas son compatibles porque solo difieren en una constante, p(x).

Vamos a suponer que la condicionada en cada clase sigue una gausiana normal de media μ. Entrenar nuestro clasificador sera encontrar la media y la variancia para cada clase. En una gausiana, el exponente es parecido a una distancia (si sigma vale uno o es la identidad es una distancia). Una vez calculemos la media, varianza, probabilidad a priori y la inversa de la varianza, sustituiremos los valores en la función discriminante:

$d_i(x) = -\frac{1}{2}ln|\Sigma_i| - \frac{1}{2}(x-\mu_i)^T\Sigma_i^{-1}(x-\mu_i) + ln P(\alpha_i)$


 


### 3.2 Implementación del clasificador


Nuestra función **fit** se encargará de calcular la probabilidad a priori (pues es independiente de X), la matriz de covarianzas y la media de cada clase. Quedando:

In [0]:
def fit_estadistico(X,y):
  unique, counts = np.unique(y,return_counts=True)
  self.labels = unique
  prob_priori = np.divide(counts,y.sum())
  ln_priori = np.log(prob_priori)
  self.ln_apriories = ln_priori
  self.means = np.array([np.mean(X[y==l],axis=0) for l in np.unique(y)])
  cov = np.array([np.cov(X[y==l] - self.means[l], rowvar=False) for l in np.unique(y)])
  det = np.array(np.linalg.det(cov))
  self.ln_determinants = np.log(det)
  self.inv_covs = np.linalg.inv(cov)

Nuestra función predict, calculará la resta entre las X y las medias y computará la función discriminante:

In [0]:
def predict_estadistico(X,y):
  mean_x0 = np.array(X - slef.means[:,np.newaxis])
  result = np.empty(((len(self.labels),np.shape(X)[0])))
  for j in range(0,np.shape(mean_x0)[0]):
    result[j] = -0.5*self.ln_determinants[j] + -0.5*(mean_x0[j].dot(self.inv_covs[j])*mean_x0[j]).sum(1) + slef.ln_apriories[j]


La funcion predict simplemente se quedará con los argumentos máximos:

In [0]:
def predict_label_estadistico(self,X):
  pred = np.argmax(result,axis=0)
  num_aciertos = (pred == y).sum()
  media_aciertos = (pred == y).mean()*100
  print(f"Numero de Aciertos es: {num_aciertos}, Porcentaje de aciertos: {media_aciertos}")

Quedando la clase completa como:

In [0]:
import numpy as np
from abc import abstractmethod

class Classifier:

    @abstractmethod
    def fit(self,X,y):
        pass

    @abstractmethod
    def predict(self,X):
        pass

class ClassifBayesiano(Classifier):
    def __init__(self):
        """Constructor de la clase
        labels: lista de etiquetas de esta clase"""
        self.ln_apriories = None
        self.means = None
        self.ln_determinants = None
        self.inv_covs = None
    def fit_estadistico(self,X,y):
        assert X.ndim == 2 and X.shape[0] == len(y)
        unique, counts = np.unique(y,return_counts=True)
        self.labels = unique
        prob_priori = np.divide(counts,y.sum())
        ln_priori = np.log(prob_priori)
        self.ln_apriories = ln_priori
        self.means = np.array([np.mean(X[y==l],axis=0) for l in np.unique(y)])
        cov = np.array([np.cov(X[y==l] - self.means[l], rowvar=False) for l in np.unique(y)])
        det = np.array(np.linalg.det(cov))
        self.ln_determinants = np.log(det)
        self.inv_covs = np.linalg.inv(cov)
        return self
    def predict_estadistico(self,X):
        assert self.means is not None, "Error: The classifier needs to be fitted. Please call fit(X, y) method."
        assert X.ndim == 2 and X.shape[1] == self.means.shape[1]
        mean_x0 = np.array(X - self.means[:,np.newaxis])
        # Nuestra variable resultado tendrá tamaño nºclases x nº datos
        result = np.empty(((len(self.labels),np.shape(X)[0])))
        # Hacemos los calculos para cada clase, multiplicamos mean_x0i por invi y luego hacemos el producto elemento por elemento con * y lo sumamos con sum(1). Esto es porque mean_x0[clase] tiene tamaños nºdatos x nºclases,
        # cov[clase] tiene tamaño nºclases x nºclases con lo cual despues de esta mult nos queda una matriz de tamaño nºdatos x nº clases. multiplicamos esta matriz elemento a elemento por mean_x0[clase] y sumamos por filas.
        for clase in range(0,np.shape(mean_x0)[0]):
          result[clase] = -0.5*self.ln_determinants[clase] + -0.5*(mean_x0[clase].dot(self.inv_covs[clase])*mean_x0[clase]).sum(1) + self.ln_apriories[clase]
        return result
    def predict_label_estadistico(self,y,grado_pertenencia):
        pred = np.argmax(grado_pertenencia,axis=0)
        num_aciertos = (pred == y).sum()
        media_aciertos = (pred == y).mean()*100
        print(f"Numero de Aciertos es: {num_aciertos}, Porcentaje de aciertos: {media_aciertos}")
    def clasfBay(X,y,test_size=0.3):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
      clasfXBayesiano = ClassifBayesiano()
      clasfXBayesiano.fit_estadistico(X_train,y_train)
      grado = clasfXBayesiano.predict_estadistico(X_test)
      clasfXBayesiano.predict_label_estadistico(y_test,grado)


### 3.3 Tabla de resultados. Discusión y comparadlos con el de distancia euclidea

#### 3.3.1 Taba de resultados

Cogiendo una muestra de test del 50% obtenemos los siguientes resultados para los tres datasets estudiados:

In [41]:
X, y  = load_iris(return_X_y=True)
print("Datos Iris Bayesiano:")
ClassifBayesiano.clasfBay(X,y,0.5)

Datos Iris Bayesiano:
Numero de Aciertos es: 71, Porcentaje de aciertos: 94.66666666666667


In [42]:
X, y  = load_wine(return_X_y=True)
print("Datos Wine Bayesiano:")
ClassifBayesiano.clasfBay(X,y,0.5)

Datos Wine Bayesiano:
Numero de Aciertos es: 84, Porcentaje de aciertos: 94.3820224719101


In [43]:
X, y  = load_breast_cancer(return_X_y=True)
print("Datos Cancer Bayesiano:")
ClassifBayesiano.clasfBay(X,y,0.5)

Datos Cancer Bayesiano:
Numero de Aciertos es: 268, Porcentaje de aciertos: 94.03508771929825


Medida |Iris | Wine | Cancer
--- |--- | --- | ---
Numero Aciertos|72 | 84 | 272
Porcentaje Aciertos|96% | 94% | 95%

#### 3.3.2 Discusión

Se puede observar que el porcentaje de aciertos es del 95% aproximadamente para las 3 bases de datos. A diferencia que en el clasificador de la distancia euclídea, este clasificador funciona fantásticamente para las 3 bases de datos y no para 2. Esto es debido a que este clasificador se basa en la estadística y no en la dispersión de las clases. No obstante, como se observará más adelante, todavía se puede mejorar ligeramente este clasificador añadiendole dos parámetros para ajustar más finamente los resultados.

## 4. Regularización


### 4.1 Explica brevemente en qué consiste la regularización y justifica su necesidad.

La regularización consiste en utilizar dos parámetros que determinan el comportamiento de nuestro clasificador. El primero será una variable booleana que determinará si las clases comparten matriz de covarianza o no y otra variable será un número entre 0-1 llamado shrinkage que consiste en reducir el ratio entre el menor autovalor y el mayor de la matriz de covarianza. Por lo tanto, nuestro clasificador bayesiano paramétrico será igual que el bayesiano calculado arriba pero teniendo en cuenta estas dos variables.

### 4.2 Implementa el clasificador Estadístico Bayesiano Paramétrico

A continuación se muestra la clase de nuestro clasificador bayesiano paramétrico

In [0]:
import numpy as np
from sklearn.covariance import ShrunkCovariance
from sklearn import preprocessing
from abc import abstractmethod
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator


class Classifier(BaseEstimator):

    @abstractmethod
    def fit(self,X,y):
        pass

    @abstractmethod
    def predict(self,X):
        pass

class ClassifBayesianoParametrico(Classifier):
    def __init__(self, share_covs=False, shrinkage=0.0):
        """Constructor de la clase
        share_covs: Indica si la matriz de covarianzas va a ser compartida entre las distintas clases.
        shrinkage: Parámetro que determina la diagonalidad de la matriz de covarianzas. Ver sklearn.covariance.ShrunkCovariance
        """
        assert 0 <= shrinkage <= 1
        self.labels = None
        self.ln_apriories = None
        self.means = None
        self.ln_determinants = None
        self.inv_covs = None
        self.share_covs = share_covs
        self.shrinkage = shrinkage
        self.scaler = preprocessing.StandardScaler()

    def fit(self, X, y):
        """Entrena el clasificador
        X: matriz numpy cada fila es un dato, cada columna una medida
        y: vector de etiquetas, tantos elementos como filas en X
        retorna objeto clasificador"""
        assert X.ndim == 2 and X.shape[0] == len(y)
        # Preprocesamos los datos de entrada
        self.scaler.fit(X)   
        X = self.scaler.transform(X)
        # Contar cuantos ejemplos hay de cada etiqueta
        unique, counts = np.unique(y,return_counts=True)
        self.labels = unique
        # Usando el contador de ejemplos de cada etiqueta, calcular el logaritmo neperiano de las probabilidades a-priori
        prob_priori = np.divide(counts,y.sum())
        self.ln_apriories = np.log(prob_priori)
        # Calcular para los ejemplos de cada clase, la media de cada una de sus características (centroide)
        self.means = np.array([np.mean(X[y==l],axis=0) for l in np.unique(y)])
        #resta = np.array([X[y==l]-self.means[l] for l in np.unique(y)])
        resta = [X[y==l] - self.means[l] for l in self.labels]
        if self.share_covs:
            # restar a cada dato la media de su clase y llamar una sola vez a ShrunkCovariance(self.shrinkage).fit(datos_menos_la_media_de_cada_clase). 
            # Restamos al dato de cada clase su centroide
            # Calcula la matriz de covarianzas empleando la clase ShrunkCovariance de sklearn
            resta = np.vstack(resta)
            resta = resta.reshape(-1, X.shape[1])
            cov1 = np.array(ShrunkCovariance(self.shrinkage).fit(resta).covariance_ )
            # La reproducimos tantas veces como número de clases
            cov = np.empty((len(unique),X.shape[1],X.shape[1]))
            for l in range(0,len(unique)):
              cov[l] = cov1
        else:
            # Calcula la matriz de covarianzas empleando la clase ShrunkCovariance de sklearn
            cov = [ShrunkCovariance(self.shrinkage).fit(resta[l]).covariance_ for l in unique]
            

        # Para cada una de las clases, calcular el logaritmo neperiano de su matriz de covarianzas (puedes emplear compresión de listas o la función map)
        self.ln_determinants = np.log(np.array(np.linalg.det(cov)))
        # Para cada una de las clases, calcular la inversa de su matriz de covarianzas (puedes emplear compresión de listas o la función map)
        self.inv_covs = np.linalg.inv(cov)
        
        return self

    def predict_proba(self, X):
        """Estima el grado de pertenencia de cada dato a todas las clases
        X: matriz numpy cada fila es un dato, cada columna una medida del vector de caracteristicas.
        Retorna una matriz, con tantas filas como datos y tantas columnas como clases tenga
        el problema, cada fila almacena los valores pertenencia de un dato a cada clase"""
        assert self.means is not None, "Error: The classifier needs to be fitted. Please call fit(X, y) method."
        assert X.ndim == 2 and X.shape[1] == self.means.shape[1]

        # Preprocesamos nuestros datos
        X = self.scaler.transform(X)
        mean_x0 = np.array(X - self.means[:,np.newaxis])
        result = np.empty(((len(self.labels),np.shape(X)[0])))
        for clase in range(0,np.shape(mean_x0)[0]):
          result[clase] = -0.5*self.ln_determinants[clase] + -0.5*(mean_x0[clase].dot(self.inv_covs[clase])*mean_x0[clase]).sum(1) + self.ln_apriories[clase]
        return result


    def predict(self, X):
        """Estima la etiqueta de cada dato. La etiqueta puede ser un entero o bien un string.
        X: matriz numpy cada fila es un dato, cada columna una medida
        retorna un vector con las etiquetas de cada dato"""
        grado_pertenencia = self.predict_proba(X)
        pred = np.argmax(grado_pertenencia,axis=0)
        return pred
        

In [0]:
def best_shrinkage_clf(X, y, k, shrinkages, share_covs):
    """
    Busca el clasificador bayesiano regularizado con el mejor shrinkage. 
    :param X: Ejemplos de la dase de datos
    :param y: Etiquetas de los ejemplos
    :param k: Número de divisiones en la validación cruzada (k-fold)
    :param shrinkages: Lista de posibles shrinkages que conforman la rejilla de búsqueda
    :param share_covs: Lista de posibles valores para share_covs que conforman la rejilla de búsqueda
    """
    from sklearn.model_selection import GridSearchCV
    cbp = ClassifBayesianoParametrico(share_covs)
    params = {'shrinkage': shrinkages, 'share_covs': share_covs}
    clf = GridSearchCV(cbp, params, n_jobs=-2, scoring='accuracy', cv=k).fit(X, y)
    best_clf = clf.best_estimator_
    # print("Srinkage scores: ", clf.cv_results_['mean_test_score'])
    result_score_mean = clf.cv_results_['mean_test_score'][clf.best_index_]
    result_score_std = clf.cv_results_['std_test_score'][clf.best_index_]
    print("\tSelected shrinkage = {}, share_covs = {}\n" \
          "\tAccuracy: {:.3f} (+/- {:.3f})".format(best_clf.shrinkage,
                                                   best_clf.share_covs,
                                                   result_score_mean,
                                                   result_score_std))
    return best_clf

## 5. Evaluación del Rendimiento

Aplicando el código proporcionado por la asignatura a los datos del iris obtenemos los siguientes datos:

In [46]:
import sklearn
from time import time
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.metrics import confusion_matrix
dataset = load_iris()
X = dataset.data
y = dataset.target
# 2. Baraja los datos para hacer validación cruzada
X, y = sklearn.utils.shuffle(X, y)
# Evaluar el clasificador de la distancia eucídea usando cross validation (k-fold=5)
k = 5
clf = ClassifEuclid()
print("Clasificador Euclídeo")
scores = cross_val_score(clf,X,y,cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# Haz una selección de modelos para buscar shrinkage del clasificador 
# estadístico paramétrico que obtiene el mejor accuracy. Usa para ello cross validation (k-fold=5)
print("Clasificador Estadístico Paramétrico")
clf = best_shrinkage_clf(X, y, k, [1,0.7,0.15,0.1,0.2,0.3,0.5,0.4], [True,False])

# Predice las muestras de la base de datos y muestra la matriz de confusión (sklearn.metrics.confusion_matrix)
clf.fit(X,y)
print(f"Matriz de Confusión:\n {confusion_matrix(y, clf.predict(X))}")

Clasificador Euclídeo
Accuracy: 95.33 (+/- 5.33)
Clasificador Estadístico Paramétrico
	Selected shrinkage = 1, share_covs = True
	Accuracy: 0.973 (+/- 0.025)
Matriz de Confusión:
 [[50  0  0]
 [ 0 47  3]
 [ 0  1 49]]


Para los datos de wine:

In [47]:
# 1. Cargar los datos de la base de datos de entrenamiento
from sklearn.datasets import load_wine
dataset = load_wine()
X = dataset.data
y = pd.factorize(dataset.target)[0]
# 2. Baraja los datos para hacer validación cruzada
X, y = sklearn.utils.shuffle(X, y)

# Evaluar el clasificador de la distancia eucídea usando cross validation (k-fold=5)
k = 5
print("Clasificador Euclídeo")
clf = ClassifEuclid()
scores = cross_val_score(clf,X,y,cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

print("Clasificador Estadístico Paramétrico")

clf = best_shrinkage_clf(X, y, k, [0.92,0.2,0.3,0.5,0.4,0.7], [True,False])

# Predice las muestras de la base de datos y muestra la matriz de confusión (sklearn.metrics.confusion_matrix)
clf.fit(X,y)
print("")
confusion_matrix(y, clf.predict(X))

Clasificador Euclídeo
Accuracy: 71.92 (+/- 24.06)
Clasificador Estadístico Paramétrico
	Selected shrinkage = 0.92, share_covs = True
	Accuracy: 1.000 (+/- 0.000)



array([[59,  0,  0],
       [ 0, 71,  0],
       [ 0,  0, 48]])

Finalmente para los datos del cancer:

In [51]:
# 1. Cargar los datos de la base de datos de entrenamiento
from sklearn.datasets import load_breast_cancer

dataset = load_breast_cancer()
X = dataset.data
y = pd.factorize(dataset.target)[0]
X, y = sklearn.utils.shuffle(X, y)

# Evaluar el clasificador de la distancia eucídea usando cross validation (k-fold=5)
k = 5
print("Clasificador Euclídeo")
clf = ClassifEuclid()
scores = cross_val_score(clf,X,y,cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

print("Clasificador Estadístico Paramétrico")
clf = best_shrinkage_clf(X, y, k, [0.71,0.3,0.5,0.4,0.7], [True,False])

# Predice las muestras de la base de datos y muestra la matriz de confusión (sklearn.metrics.confusion_matrix)
clf.fit(X,y)
print("")
confusion_matrix(y, clf.predict(X))

Clasificador Euclídeo
Accuracy: 89.80 (+/- 6.27)
Clasificador Estadístico Paramétrico
	Selected shrinkage = 0.71, share_covs = True
	Accuracy: 0.961 (+/- 0.016)



array([[192,  20],
       [  1, 356]])

Analizando los resultados obtenidos podemos ver como el clasificador estadístico paramétrico arroja mejores resultados para los datasets del iris y wine pero peores para el del cancer. Cabe mencionar el 100% de acierto del clasificador paramétrico en los datos del wine mientras que el euclídeo solo es capaz de acertar un 72% de las veces.

## 6. Aplicación en un caso real de reconocimiento de texto



Para esta aplicación utilizaremos el clasificador euclídeo. Utilizaremos este clasificador porque creo que las dispersión de las clases es pequeña en relación a la distancia entre ella, o al menos en la mayoría de los casos. Para empezar, eliminaremos ciertos carácteres que no queramos que nuestro clasificador detecte, esto, obviamente, solo es válido para el texto de prueba, pero puede ser un buen primer paso para, más adelante, depurarlo un poco más. Eliminaremos los carácteres 'w','x','k' y 'z' así como los números '5','7' y '8'. A parte de quitar estoss carácteres, añadiremos algunos otros como á y ú y - y :. Más adelante veremos que los carácteres que detecte como á y ú los mapearemos a las letras a y u. También, para nuestros datos de entrenamiento, utilizaremos ejemplos sin ruido y ejemplos con ruido en una relación 1:2 aproximadamente siendo los ejemplos con ruido los más numerosos. Con solo estos cambios ya somos capaces de obtener un porcentaje de aciertos del 80%.

La segunda tanda de cambios que realizaremos tendrán que ver en como procesamos los dígitos detectados. Lo primero que haremos será aumentar los píxeles (a por ejemplo 3000 pixeles) de la foto para que se vea con más claridad y estén más claros los símbolos. Agregaremos un filtro que, a partir de un threshold, considerará pixeles que sean medio oscuros o medio claros como oscuros y claros. Modificaremos el split a 1 (antes 1.5) y cambiaremos un poco el bitmap para que sea más alargado (28x25) con todos estos cambios obtenemos un porcentaje de aciertos del 93.4% (de media), para casos de prueba que no son el proporcionado por el profesor. 

## 7. Conclusiones

A lo largo de esta práctica hemos visto como desarrollar distintos clasificadores y como evaluar el rendimiento de los mismos. Finalmente, hemos aplicado los conocimientos adquiridos a un caso real. Esta práctica muestra como clasificadores muy sencillos son capaces de resolver problemas de clasificacion con cierta facilidad llegando incluso a porcentajes de acierto superiores al 90%.